In [ ]:
import pandas as pd
import glob
import os
import numpy as np
import random

import torch
from torch import nn
from torch.utils.data import DataLoader
from itertools import combinations,permutations

torch.manual_seed(99)
np.random.seed(99)
torch.cuda.empty_cache()
import torch.nn.functional as F

import numpy as np
import torch
from torch import nn
from torch.autograd import Variable
from torch.nn import functional as tf

from enum import Enum
from typing import List

import pandas as pd
import pkg_resources
import torch
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from torch.utils.data import TensorDataset, DataLoader

import networkx as nx
from tqdm import tqdm
from torch_geometric.nn import GCNConv, GATConv
import plotly.graph_objects as go
from scipy.sparse import coo_matrix
from geopy.distance import distance

In [ ]:
# frame = pd.read_csv("C:/Users/psheth5/STCD-RL/data/data/watershed_avg.csv")
frame = pd.read_csv("C:/Users/psheth5/STCD-RL/data/data/StandardizedData.csv")
n_features = frame.shape[1]
frame

In [ ]:
# import numpy as np

# adj_mat = np.load('C:/Users/psheth5/STCD-RL/data/data/watershed.npy')

adj_mat = np.load('ElevationAdjacency.npy')


In [ ]:
# # distances = np.load('C:/Users/psheth5/STCD-RL/distance.npy')
# # np.fill_diagonal(distance, np.inf)
# # distances[np.isfinite(distance)] = 1   # replace finite values with 1
# # distances[~np.isfinite(distance)] = 0   # replace infinite values with 0
# # distances

# list_of_coords_str = frame.columns # your list of 3129 latitude longitude values
# list_of_coords = [tuple(map(float, c.split()[::-1])) for c in list_of_coords_str]
# adj_matrix_list = np.zeros((len(list_of_coords), len(frame.columns)))
# for i, c1 in enumerate(list_of_coords):
#     for j, c2 in enumerate(list_of_coords):
#         if c2[0] > c1[0] and distance(c1, c2).km > 0:
#             adj_matrix_list[i][j] = 1

In [ ]:
# # adj_mat = np.load('C:/Users/psheth5/STCD-RL/data/data/watershed.npy')
# coo = coo_matrix(adj_matrix_list, dtype = "int8")
coo = coo_matrix(adj_mat, dtype = "int8")
row = torch.from_numpy(coo.row.astype(np.int64)).to(torch.long)
col = torch.from_numpy(coo.col.astype(np.int64)).to(torch.long)
edge_index = torch.stack([row, col], dim=0)

In [ ]:
# np.save("data\\data\\adj_mat_new.npy", adj_matrix_list)

In [ ]:
import numpy as np
import torch

def filter_edge_index(edge_index, coords):
    filtered_index = []
    for i in range(edge_index.shape[1]):
        u = edge_index[0][i]
        v = edge_index[1][i]
        # check if u is above v
        if torch.tensor(coords[u][0]) > torch.tensor(coords[v][0]) and u != v:
            filtered_index.append([u, v])
    filtered_index = np.array(filtered_index).T
    return filtered_index

In [ ]:
# edge_index = filter_edge_index(edge_index, list_of_coords)
# edge_index = torch.from_numpy(edge_index.astype(np.int64)).to(torch.long)

In [ ]:
# create a mapping function
def map_values(value, value_range):
    """Maps a value from the input range to the output range."""
    input_min, input_max = value_range
    output_min, output_max = (0, 99)
    return torch.floor(((value - input_min) / (input_max - input_min)) * (output_max - output_min) + output_min).long()


def reverse_map_values(mapped_value, value_range):
    """Maps a value from the output range to the input range."""
    input_min, input_max = value_range
    output_min, output_max = (0, 99)
    return torch.floor(((mapped_value - output_min) / (output_max - output_min)) * (input_max - input_min) + input_min).long()

In [ ]:
class Tasks(Enum):
    prediction = "prediction"
    reconstruction = "reconstruction"


class TimeSeriesDataset(object):
    def __init__(self, task: Tasks, data_path: str, categorical_cols: List[str], index_col: str, target_col: str,
                 seq_length: int, batch_size: int, prediction_window: int = 1):
        """
        :param task: name of the task
        :param data_path: path to datafile
        :param categorical_cols: name of the categorical columns, if None pass empty list
        :param index_col: column to use as index
        :param target_col: name of the targeted column
        :param seq_length: window length to use
        :param batch_size:
        :param prediction_window: window length to predict
        """
        self.task = task.value

        

        # data_path = pkg_resources.resource_filename("tsa", data_path)
        self.data = pd.read_csv(data_path, index_col=index_col)[0:1460]
        # a  = [self.data.columns[0]]
        # a.extend(self.data.columns[-250:])
        # self.data = self.data[a]
        self.categorical_cols = categorical_cols
        # self.numerical_cols = list(set(self.data.columns) - set(categorical_cols) - set(target_col))
        self.numerical_cols = self.data.columns
        print(len(self.numerical_cols))
        self.target_col = target_col

        self.seq_length = seq_length
        self.prediction_window = prediction_window
        self.batch_size = batch_size

        transformations = [("scaler", StandardScaler(), self.numerical_cols)]
        if len(self.categorical_cols) > 0:
            transformations.append(("encoder", OneHotEncoder(), self.categorical_cols))
        self.preprocessor = ColumnTransformer(transformations, remainder="passthrough")

        if self.task == "prediction":
            self.y_scaler = StandardScaler()

    def preprocess_data(self):
        """Preprocessing function"""
        X = self.data#.drop(self.target_col, axis=1,inplace=False)
        # y = self.data[self.target_col]

        X_train, X_test = train_test_split(X,train_size=0.8, shuffle=False)
        X_train = self.preprocessor.fit_transform(X_train)
        X_test = self.preprocessor.transform(X_test)

        # if self.task == "prediction":
        #     y_train = self.y_scaler.fit_transform([y_train])
        #     y_test = self.y_scaler.fit_transform([y_test])
        #     return X_train, X_test, y_train.reshape(-1,1), y_test.reshape(-1,1)
        return X_train, X_test, None,None

    def frame_series(self, X, y=None):
        """
        Function used to prepare the data for time series prediction
        :param X: set of features
        :param y: targeted value to predict
        :return: TensorDataset
        """
        nb_obs, nb_features = X.shape
        features, target, y_hist = [], [], []

        for i in range(1, nb_obs - self.seq_length - self.prediction_window):
            features.append(torch.FloatTensor(X[i:i + self.seq_length, :]).unsqueeze(0))

            if self.task == "prediction":
                # lagged output used for prediction
                y_hist.append(torch.FloatTensor(y[i - 1:i + self.seq_length - 1]).unsqueeze(0))
                # shifted target
                target.append(torch.FloatTensor(y[i + self.seq_length:i + self.seq_length + self.prediction_window]).unsqueeze(0))
            else:
                y_hist.append(torch.FloatTensor(X[i - 1: i + self.seq_length - 1, :]).unsqueeze(0))
                target.append(
                    torch.FloatTensor(X[i + self.seq_length:i + self.seq_length + self.prediction_window, :]))

        features_var = torch.cat(features)
        y_hist_var = torch.cat(y_hist)
        target_var = torch.cat(target)

        return TensorDataset(features_var, y_hist_var, target_var)

    def get_loaders(self):
        """
        Preprocess and frame the dataset

        :return: DataLoaders associated to training and testing data
        """
        X_train, X_test, y_train, y_test = self.preprocess_data()
        nb_features = X_train.shape[1]

        train_dataset = self.frame_series(X_train, y_train)
        test_dataset = self.frame_series(X_test, y_test)

        train_iter = DataLoader(train_dataset, batch_size=self.batch_size, shuffle=True, drop_last=True)
        test_iter = DataLoader(test_dataset, batch_size=self.batch_size, shuffle=True, drop_last=True)
        return train_iter, test_iter, nb_features

    def invert_scale(self, predictions):
        """
        Inverts the scale of the predictions
        """
        if isinstance(predictions, torch.Tensor):
            predictions = predictions.numpy()

        if predictions.ndim == 1:
            predictions = predictions.reshape(-1, 1)

        if self.task == "prediction":
            unscaled = self.y_scaler.inverse_transform(predictions)
        else:
            unscaled = self.preprocessor.named_transformers_["scaler"].inverse_transform(predictions)
        return torch.Tensor(unscaled)

In [ ]:
task = Tasks('reconstruction')
data1 = TimeSeriesDataset(task=task,data_path="C:/Users/psheth5/STCD-RL/data/data/StandardizedData.csv",categorical_cols=[],index_col=None,target_col=frame.columns[0],seq_length=30,batch_size=256,prediction_window=1)
train_iter, test_iter, nb_features = data1.get_loaders()

In [ ]:
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = 'cpu'


def init_hidden(x: torch.Tensor, hidden_size: int, num_dir: int = 1, xavier: bool = True):
    """
    Initialize hidden.

    Args:
        x: (torch.Tensor): input tensor
        hidden_size: (int):
        num_dir: (int): number of directions in LSTM
        xavier: (bool): wether or not use xavier initialization
    """
    if xavier:
        return nn.init.xavier_normal_(torch.zeros(num_dir, x.size(0), hidden_size)).to(device)
    return Variable(torch.zeros(num_dir, x.size(0), hidden_size)).to(device)


###########################################################################
################################ ENCODERS #################################
###########################################################################
class AttnEncoder(nn.Module):
    def __init__(self, hidden_size_encoder,seq_len,denoising,directions,input_size,edge_index,spatial,use_spatial=False):
        """
        Initialize the network.

        Args:
            config:
            input_size: (int): size of the input
        """
        super(AttnEncoder, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size_encoder
        self.seq_len = seq_len
        self.add_noise = denoising
        self.gcn_e = GCNConv(in_channels=-1, out_channels=self.hidden_size)
        self.directions = directions
        self.lstm = nn.LSTM(
            input_size=self.input_size,
            hidden_size=self.hidden_size,
            num_layers=1
        )
        self.attn = nn.Linear(
            in_features=2 * self.hidden_size + self.seq_len,
            out_features=1
        )
        self.softmax = nn.Softmax(dim=1)
        self.edge_index=edge_index
        self.use_spatial = use_spatial
        self.spatial_mat = spatial

    def forward(self, input_data: torch.Tensor,sample:torch.Tensor):
        """
        Forward computation.

        Args:
            input_data: (torch.Tensor): tensor of input data
        """
        spatial = self.spatial_mat[sample]
        spatial = spatial.to(device)
        h_t, c_t = (init_hidden(input_data, self.hidden_size, num_dir=self.directions),
                    init_hidden(input_data, self.hidden_size, num_dir=self.directions))

        # apply the mapping function to the sampled tensor
        a = self.edge_index[0][sample]
        b = self.edge_index[1][sample]
        sample_edge_index = torch.stack([a, b], dim=0)
        mapped_arr = map_values(sample_edge_index, (sample_edge_index.min(), sample_edge_index.max()))
        
        attentions, input_encoded = (Variable(torch.zeros(input_data.size(0), self.seq_len, self.input_size)),
                                     Variable(torch.zeros(input_data.size(0), self.seq_len, self.hidden_size)))

        if self.add_noise and self.training:
            input_data += self._get_noise(input_data).to(device)
        
        for t in range(self.seq_len):
            x = torch.cat((h_t.repeat(self.input_size, 1, 1).permute(1, 0, 2),
                           c_t.repeat(self.input_size, 1, 1).permute(1, 0, 2),
                           input_data.permute(0, 2, 1).to(device)), dim=2).to(
                device)  # bs * input_size * (2 * hidden_dim + seq_len)

            e_t = self.attn(x.view(-1, self.hidden_size * 2 + self.seq_len))  # (bs * input_size) * 1
            a_t = self.softmax(e_t.view(-1, self.input_size)).to(device)  # (bs, input_size)
            weighted_input = torch.mul(a_t, input_data[:, t, :].to(device))  # (bs * input_size)
            self.lstm.flatten_parameters()
            _, (h_t, c_t) = self.lstm(weighted_input.unsqueeze(0), (h_t, c_t))

            if(self.use_spatial):
                h_t = self.gcn_e(h_t,mapped_arr.to(device))
            input_encoded[:, t, :] = h_t
            attentions[:, t, :] = a_t




        return attentions, input_encoded


###########################################################################
################################ DECODERS #################################
###########################################################################

class AttnDecoder(nn.Module):
    def __init__(self, seq_len,hidden_size_encoder,hidden_size_decoder,output_size,edge_index):
        """
        Initialize the network.

        Args:
            config:
        """
        super(AttnDecoder, self).__init__()
        self.seq_len = seq_len
        self.encoder_hidden_size = hidden_size_encoder
        self.decoder_hidden_size = hidden_size_decoder
        self.out_feats = output_size

        self.gcn_d = GCNConv(in_channels=-1, out_channels=self.decoder_hidden_size)

        self.attn = nn.Sequential(
            nn.Linear(2 * self.decoder_hidden_size + self.encoder_hidden_size, self.encoder_hidden_size),
            nn.Tanh(),
            nn.Linear(self.encoder_hidden_size, 1)
        )
        self.lstm = nn.LSTM(input_size=self.out_feats, hidden_size=self.decoder_hidden_size)
        self.fc = nn.Linear(self.encoder_hidden_size + self.out_feats, self.out_feats)
        self.fc_out = nn.Linear(self.decoder_hidden_size + self.encoder_hidden_size, self.out_feats)
        self.fc.weight.data.normal_()
        self.edge_index = edge_index
    

    def forward(self, input_encoded: torch.Tensor, y_history: torch.Tensor, sample: torch.Tensor):
        """
        Perform forward computation.

        Args:
            input_encoded: (torch.Tensor): tensor of encoded input
            y_history: (torch.Tensor): shifted target
        """
        h_t, c_t = (init_hidden(input_encoded, self.decoder_hidden_size), init_hidden(input_encoded, self.decoder_hidden_size))
        context = Variable(torch.zeros(input_encoded.size(0), self.encoder_hidden_size))

        a = self.edge_index[0][sample]
        b = self.edge_index[1][sample]
        sample_edge_index = torch.stack([a, b], dim=0)
        mapped_arr = map_values(sample_edge_index, (sample_edge_index.min(), sample_edge_index.max()))

        for t in range(self.seq_len):
            x = torch.cat((h_t.repeat(self.seq_len, 1, 1).permute(1, 0, 2),
                           c_t.repeat(self.seq_len, 1, 1).permute(1, 0, 2),
                           input_encoded.to(device)), dim=2)
            x = tf.softmax(
                self.attn(
                    x.view(-1, 2 * self.decoder_hidden_size + self.encoder_hidden_size)
                ).view(-1, self.seq_len),
                dim=1)
            context = torch.bmm(x.unsqueeze(1), input_encoded.to(device))[:, 0, :]  # (batch_size, encoder_hidden_size)
            y_tilde = self.fc(torch.cat((context.to(device), y_history[:, t].to(device)),
                                        dim=1))  # (batch_size, out_size)            
            self.lstm.flatten_parameters()
            _, (h_t, c_t) = self.lstm(y_tilde.unsqueeze(0), (h_t, c_t))
            h_t = self.gcn_d(h_t,mapped_arr.to(device))
        

        return self.fc_out(torch.cat((h_t[0], context.to(device)), dim=1))  # predicting value at t=self.seq_length+1


class AutoEncForecast(nn.Module):
    def __init__(self, input_att,temporal_att,hidden_size_encoder,seq_len,denoising,directions,hidden_size_decoder,input_size,output_size,sample_size,edge_index,spatial_matrix,use_spatial=False):
        """
        Initialize the network.

        Args:
            config:
            input_size: (int): size of the input
        """
        super(AutoEncForecast, self).__init__()
        self.encoder = AttnEncoder(hidden_size_encoder,seq_len,denoising,directions,input_size,edge_index,spatial_matrix,use_spatial).to(device) if input_att else \
            Encoder(hidden_size_encoder, seq_len, input_size,edge_index).to(device)
        self.decoder = AttnDecoder(seq_len,hidden_size_encoder,hidden_size_decoder,output_size,edge_index).to(device) if temporal_att else Decoder(seq_len,hidden_size_decoder,output_size,edge_index).to(device)

        self.fc1 = nn.Linear(512,1)#.cuda()
        self.tanh = nn.Tanh()#.cuda()
        self.sigmoid = nn.Sigmoid()
        self.sample_size = sample_size

    def calcPerm(self,l: list, m: int):
        for i in permutations(l, m):
            yield list((i))
    
    def forward(self, encoder_input: torch.Tensor, y_hist: torch.Tensor, sample: torch.Tensor ,return_attention: bool = False):
        """
        Forward computation. encoder_input_inputs.

        Args:
            encoder_input: (torch.Tensor): tensor of input data
            y_hist: (torch.Tensor): shifted target
            return_attention: (bool): whether or not to return the attention
        """
        attentions, encoder_output = self.encoder(encoder_input, sample)   

        outputs = self.decoder(encoder_output, y_hist.float(), sample)
        g = torch.zeros(outputs.shape[1],outputs.shape[1])#.cuda()
        outputs = outputs.transpose(1,0)
        idx1 = 0
        idx2 = 0
        for i,j in self.calcPerm(outputs, 2):
                a = self.tanh(self.fc1(torch.cat((i,j))))
                g[idx1,idx2] = a
                if((idx2+1)%self.sample_size):
                    idx2+=1
                else:
                    idx2 = 0
                    idx1+= 1
        if return_attention:
            return outputs, attentions,g
        return outputs,g

In [ ]:
# target_index = 252
target_index = 0

In [ ]:
# distance = np.load('C:/Users/psheth5/STCD-RL/data/data/watershed.npy')
# distance = distance[:, target_index]

# spatial_matrix = []

# for d in distance:
#     if np.isfinite(d):
#         spatial_matrix.append(1 / (d + 1))
#     else:
#         spatial_matrix.append(0)

# spatial_matrix1 = torch.FloatTensor(spatial_matrix).cuda()
# torch.max(spatial_matrix1)

spatial_matrix1 = torch.FloatTensor(adj_mat).cuda()
torch.max(spatial_matrix1)

In [ ]:
sampling_size = 100
model = AutoEncForecast(input_att=True,temporal_att=True,hidden_size_encoder=64,seq_len=30,denoising=False,directions=1,hidden_size_decoder=64,input_size=sampling_size,output_size=sampling_size,sample_size=sampling_size,edge_index=edge_index,spatial_matrix = spatial_matrix1,use_spatial=True).to(device)

In [ ]:
import random
def update_graph(sample,inferred_cg,g):
    for i in range(len(sample)):
        for j in range(len(sample)):
            inferred_cg[sample[i],sample[j]] += g[i,j].cpu()
    return inferred_cg

def update_attention(sample,att,attention):
    temp = torch.mean(att,axis=1)
    for i in range(len(sample)):
        for j in range(len(sample)):
            attention[sample[i],sample[j]] += temp[i,j].cpu()
    return attention

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

def get_num_cycles(gra):
    # gra = gra.cpu().detach().numpy()
    G = nx.from_numpy_matrix(gra)
    H = G.to_directed()
    try:
        return torch.FloatTensor([len(list(nx.find_cycle(H, orientation="original")))])#.cuda()
    except:
        return 0

def is_dag(gr):
    # gr = gr.cpu().detach().numpy()
    G = nx.from_numpy_matrix(gr)
    return int(nx.is_directed_acyclic_graph(G))

In [ ]:
## helper functions
def calc_discounted_rewards(rewards, gamma):
    ''' 
    Simple implementation for better understanding
    gets rewards of an entire episode and calculates R_t for every t
    '''
    
    returns = []
    
    for t in range(len(rewards)):
        ret = 0
        
        for t_p in range(t, len(rewards)):
            ret += gamma ** (t_p - t) * rewards[t_p]
            
        returns.insert(0, ret)
        
    return returns

In [ ]:
random.seed(26)
np.random.seed(26)
torch.manual_seed(26)
torch.cuda.manual_seed(26)

optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)
global_step, logging_loss = 0, 0.0
train_loss = 0.0
reg1= True
reg2= True
reg_factor1= 1e-8
reg_factor2= 1e-8
gradient_accumulation_steps = 1
max_grad_norm = 0.1
logging_steps = 100
criterion = nn.BCELoss()
crit_mse = nn.MSELoss()
crit_rss = nn.MSELoss(reduction='none')
crit_nll = nn.NLLLoss()
save_steps = 5000
eval_during_training = False
output_dir = "Models"
lrs_step_size=5000
do_eval=True
reg_factor = 1
BASELINE_REWARD = 20
breakout = 0
inferred_cg = np.zeros((n_features,n_features))
attention = np.zeros((n_features,n_features))
min_loss = 1e+5
for epoch in tqdm(range(5), unit="epoch"):
    attentions = []
    outputs = []
    total_loss = 0
    loss_1 = []
    loss_2 = []
    for i, batch in tqdm(enumerate(train_iter), total=len(train_iter), unit="batch"):
        original_list = list(range(3129))
        while(len(original_list)!=0):
            sampled_list = []
            optimizer.zero_grad()
            model.train()
            feature, y_hist, target = batch
            try:
                sample = random.sample(original_list, sampling_size)
            except:
                sample = random.sample(original_list,len(original_list))
                sample.extend(random.sample(range(feature.shape[2]),sampling_size-len(original_list)-1))
                sample.append(0)
            # add the sampled elements to the sampled list
            sampled_list.extend(sample)
            # remove the sampled elements from the original list
            original_list = [element for element in original_list if element not in sample]
            feature = feature[:,:,sample]
            y_hist = y_hist[:,:,sample]
            target = target[:,sample]
            
            output,att,g = model(feature.to(device), y_hist.to(device),sample,return_attention=True)
            temp = torch.mean(att,axis=1)
            output = output.reshape(output.shape[1],output.shape[0])
            inferred_cg = update_graph(sample,inferred_cg,g)
            inf_cg = torch.FloatTensor(inferred_cg)
            attention = update_attention(sample,att,attention)
            num_params = count_parameters(model)
            #target = target.cuda()
            loss2 = crit_mse(output,target)

            n = torch.FloatTensor([feature.shape[0]])#.cuda()
            cyc = torch.trace(torch.matrix_exp(inf_cg*inf_cg)) - 3129
            bic = n*3129*(torch.log(torch.sum(crit_rss(output,target))/(3129*n))) + torch.sum(inf_cg)*torch.log(n)
            loss = -1*(loss2 + reg_factor1*cyc + reg_factor2*bic)
            loss_2.append(loss2)
            loss_1.append(loss.item())
            if(len(original_list)%1000==0):
                print("The number of nodes sampled are ", 3129 - len(original_list))
    loss_tot = 0
    loss_21 = calc_discounted_rewards(loss_2,0.98)
    for indexe in range(len(loss_21)):
        loss_tot += loss_1[indexe] * torch.log(loss_21[indexe])
    loss_tot = (loss_tot - BASELINE_REWARD) / len(loss_21)
    loss_tot.backward()
    optimizer.step()
    total_loss +=loss_tot.item()
    torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
    if(round(total_loss/(i+1),2)<=round(min_loss/(i+1),2)):
        min_loss = total_loss
        breakout = 0
        torch.save(model.state_dict(),os.getcwd()+"/Models/MyModel_GCN_FullModel_"+str(epoch))
    else:
        breakout+=1
    print("Loss is ", total_loss, " for epoch ", epoch)
    print(np.where(attention[target_index,:]!=0),len(np.where(attention[target_index,:]!=0)[0]),get_num_cycles(inferred_cg),is_dag(inferred_cg))

In [ ]:
def plot_graph(a_new,fname):
    latitude = []
    longitude = []
    for j,i in enumerate(a_new):
            latitude.append(float(i.split()[0]))
            longitude.append(float(i.split()[1]))
    fig = go.Figure()
    fig.add_trace(go.Scattermapbox(
        mode = "markers",
        lat = longitude,
        lon = latitude,
        marker = { 'color':'red',
                    'size': 10}
    ))

    fig.update_layout(margin={'l':0,'t':0,'b':0,'r':0},
                    mapbox={
                        'center':{'lon':139,'lat':36.5},
                        'style': "stamen-terrain",
                        'zoom' : 4.5},
                    width=1600,
                    height=900,)
    fig.show()
    fig.write_html(fname)

In [ ]:
at1 = frame.columns[inferred_cg[target_index,:].argsort()[-100:]]
plot_graph(at1,"mapTestFT1.html")

In [ ]:
np.save("attention.npy",attention)
np.save("inferred_cg.npy",inferred_cg)